In [22]:
import os, openai, json
import keys, modify_spec
os.environ["OPENAI_API_KEY"] = keys.OPEN_AI_KEY
os.environ["HELIUS_API_KEY"] = keys.HELIUS_KEY
openai.api_key = keys.OPEN_AI_KEY
from langchain.chains.openai_functions.openapi import get_openapi_chain

In [23]:
# with open("local_solana_openapi.yaml") as f:
#     raw_solana_api_spec = yaml.load(f, Loader=yaml.Loader)
# solana_api_spec = reduce_openapi_spec(raw_solana_api_spec)
# print(len(solana_api_spec.endpoints))
# # pretty print the first endpoint
# print(json.dumps(solana_api_spec.endpoints[0], indent=2))
# # print(type(solana_api_spec.endpoints[0]))

In [26]:
# f = open("local_solana_openapi.yaml")
chain = get_openapi_chain("local_solana_openapi.yaml", verbose = True)


Attempting to load an OpenAPI 3.0.2 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


In [28]:
chain.run("Could you tell me all the assets owned by owner 8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625 ?")



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
Human: Use the provided API's to respond to this user query:

Could you tell me all the assets owned by owner 8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625 ?

> Finished chain.


> Entering new  chain...
Calling endpoint query_assets_by_owner with arguments:
{
  "json": {
    "address": "8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625"
  }
}

KeyError: 'params'